In [1]:
import keras
import numpy as np
from keras import backend as K
from random import shuffle
from keras.applications.vgg16 import VGG16
from keras.models import Model, Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D, Input, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.callbacks import ModelCheckpoint
from keras.optimizers import SGD

print(keras.__version__)

Using TensorFlow backend.


2.2.4


In [2]:
from keras import backend as k
import tensorflow as tf

config = tf.ConfigProto()
config.gpu_options.allow_growth = True

k.tensorflow_backend.set_session(tf.Session(config=config))

In [3]:
batch_size = 64
IMAGE_SIZE = 64
epochs = 100
num_classes = 5

In [4]:
# train = np.load("/home/cudacenter/DR/DenseNet_Finetune/Train64x64.npy")
# X_train = np.array([i[0] for i in train]).reshape(-1, IMAGE_SIZE, IMAGE_SIZE, 3)
# Y_train = np.array([i[1] for i in train])

# del train
# print("Train Loaded")

In [5]:
# train = np.load("/home/cudacenter/DR/DenseNet_Finetune/Val64x64.npy")
train = np.load("F:/DR/Final_NPYs_6March/TestFinal6Mx64x64.npy")

X_test = np.array([i[0] for i in train]).reshape(-1, IMAGE_SIZE, IMAGE_SIZE, 3)
Y_test = np.array([i[1] for i in train])

del train
print("Test Loaded")

Test Loaded


In [6]:
# unique_elements, counts_elements = np.unique(Y_train, return_counts=True)
# print("Frequency of unique values of the said array:")
# print(np.asarray((unique_elements, counts_elements)))

In [7]:
unique_elements, counts_elements = np.unique(Y_test, return_counts=True)
print("Frequency of unique values of the said array:")
print(np.asarray((unique_elements, counts_elements)))

Frequency of unique values of the said array:
[[  0   1   2   3   4]
 [150 150 150 150 150]]


In [8]:
# X_train = X_train.astype("float32")
X_test = X_test.astype("float32")

# X_train /= 255
X_test /= 255

# Y_train = keras.utils.to_categorical(Y_train,num_classes)
Y_test = keras.utils.to_categorical(Y_test,num_classes)

In [9]:
model = Sequential()
model.add(Conv2D(96, 3, strides=1, input_shape=(64,64,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=3, strides=2))
# model.add(Dropout(0.3))

model.add(Conv2D(256, 5, strides=1))
model.add(ZeroPadding2D(2))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=3, strides=2))
# model.add(Dropout(0.3))

model.add(Conv2D(384, 3, strides=1))
model.add(ZeroPadding2D(1))
model.add(Activation('relu'))
# model.add(Dropout(0.3))

model.add(Conv2D(384, 3, strides=1))
model.add(ZeroPadding2D(1))
model.add(Activation('relu'))
# model.add(Dropout(0.3))


model.add(Conv2D(256, 3, strides=1))
model.add(ZeroPadding2D(1))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=3, strides=2))
# model.add(Dropout(0.3))


model.add(Flatten())

model.add(Dense(4096))
model.add(Activation('relu'))
# model.add(Dropout(0.3))

model.add(Dense(4096))
model.add(Activation('relu'))
# model.add(Dropout(0.3))

model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [10]:
model.summary()
#64x64

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 62, 62, 96)        2688      
_________________________________________________________________
activation_1 (Activation)    (None, 62, 62, 96)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 30, 96)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 26, 26, 256)       614656    
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 30, 30, 256)       0         
_________________________________________________________________
activation_2 (Activation)    (None, 30, 30, 256)       0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 256)       0         
__________

In [11]:
# model.summary()
# #224x224

In [12]:
model.load_weights("load7.h5")

In [13]:
## CREATE DIRECTORY FIRST /home/cudacenter/DR/FINAL_05Feb/Pretrained/DenseNet_Weights_t1
sgd = SGD(lr=1e-3, decay=1e-6, momentum = 0.9, nesterov=True)
# checkpoint = ModelCheckpoint('/home/cudacenter/DR/AlexNet/alex64-{epoch:03d}-.h5',verbose = 1,monitor = 'acc',save_best_only = True,mode = "auto")
model.compile(loss=keras.losses.categorical_crossentropy, optimizer=sgd, metrics=['accuracy'])

In [14]:
# model.fit(X_train, Y_train,batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(X_test, Y_test), callbacks=[checkpoint])

In [15]:
# model.load_weights("/home/cudacenter/DR/DenseNet_Finetune/64x64x2/DN121x64-022-.h5")
score = model.evaluate(X_test,Y_test,verbose=1)
print("Test Loss : ",score[0])
print("Test Accuracy : ",score[1])
Y_pred = model.predict(X_test,verbose = 1)

750/750 [==============================] - 13s 17ms/step
Test Loss :  1.4505118192632993
Test Accuracy :  0.7453333328564962
750/750 [==============================] - 3s 5ms/step


In [16]:
# model.save_weights('/home/cudacenter/DR/AlexNet/DR_weights_{}_AlexF64.h5'.format(score[1]))

In [17]:
# print(Y_pred)
from sklearn.metrics import confusion_matrix

conf_mat = confusion_matrix(Y_test.argmax(axis = 1), Y_pred.argmax(axis = 1))
print(conf_mat)

[[140   1   9   0   0]
 [ 46  97   7   0   0]
 [ 40   2 104   3   1]
 [ 19   1  23 103   4]
 [ 20   1  12   2 115]]


In [18]:
sens = (float)(conf_mat[0,0]) / (float)(6000) 
print(sens)
sens = (float)(conf_mat[1,1]) / (float)(6000) 
print(sens)
sens = (float)(conf_mat[2,2]) / (float)(6000) 
print(sens)
sens = (float)(conf_mat[3,3]) / (float)(6000) 
print(sens)
sens = (float)(conf_mat[4,4]) / (float)(6000) 
print(sens)

0.023333333333333334
0.016166666666666666
0.017333333333333333
0.017166666666666667
0.019166666666666665
